In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    


In [2]:
# Read Data
usage = pd.read_pickle('../data/pickle/usage.pickle')

df = usage
del usage

df.head()

,ACT_CLICK_DEPTH,ACT_DWELL_TIME_VISIT_MIN,ECR_ID,LOY_DWELL_TIME_USER_MIN,LOY_RETURN_RATE,POP_ACTIVE_USERS,POP_PAGE_VIEWS,POP_TIME_SPENT_HRS,POP_VISITS,PROD_NAME,REPORT_AGG,REPORT_DT
0,5.30,2.87,NaN,3.15,1.10,29190,169893,1532.52,32058,SCIVAL,MONTH,2017-12-01
1,5.21,3.02,NaN,3.39,1.12,31390,183421,1772.58,35211,SCIVAL,MONTH,2018-01-01
2,5.22,2.97,NaN,3.32,1.12,28384,165575,1568.61,31742,SCIVAL,MONTH,2018-02-01
3,5.25,3.39,NaN,3.81,1.12,32226,190321,2046.88,36237,SCIVAL,MONTH,2018-03-01
4,5.34,3.36,NaN,3.76,1.12,31703,189726,1988.12,35546,SCIVAL,MONTH,2018-04-01


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown', 'NaT'], np.nan, inplace=True)
df

,ACT_CLICK_DEPTH,ACT_DWELL_TIME_VISIT_MIN,ECR_ID,LOY_DWELL_TIME_USER_MIN,LOY_RETURN_RATE,POP_ACTIVE_USERS,POP_PAGE_VIEWS,POP_TIME_SPENT_HRS,POP_VISITS,PROD_NAME,REPORT_AGG,REPORT_DT
0,5.30,2.87,NaN,3.15,1.10,29190,169893,1532.52,32058,SCIVAL,MONTH,2017-12-01
1,5.21,3.02,NaN,3.39,1.12,31390,183421,1772.58,35211,SCIVAL,MONTH,2018-01-01
2,5.22,2.97,NaN,3.32,1.12,28384,165575,1568.61,31742,SCIVAL,MONTH,2018-02-01
3,5.25,3.39,NaN,3.81,1.12,32226,190321,2046.88,36237,SCIVAL,MONTH,2018-03-01
4,5.34,3.36,NaN,3.76,1.12,31703,189726,1988.12,35546,SCIVAL,MONTH,2018-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...
2815339,0.00,0.00,ECR-474126,0.00,0.00,0,0,0.00,0,SCIENCEDIRECT,MONTH,2018-08-01
2815340,0.00,0.00,ECR-813986,0.00,0.00,0,0,0.00,0,SCIENCEDIRECT,MONTH,2018-03-01
2815341,0.00,0.00,ECR-813986,0.00,0.00,0,0,0.00,0,SCIENCEDIRECT,MONTH,2018-08-01
2815342,0.00,0.00,ECR-813986,0.00,0.00,0,0,0.00,0,SCIENCEDIRECT,MONTH,2018-10-01


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

0.00    1187769
1.00      41838
2.00      22926
3.00      15477
4.00      11583
         ...   
4.39          3
1.70          3
7.01          3
3.63          3
3.34          3
Name: ACT_CLICK_DEPTH, Length: 248267, dtype: int64
0.00     1176183
0.02        4698
0.05        2544
0.03        2412
0.01        1635
          ...   
11.86          3
3.09           3
12.97          3
2.06           3
5.19           3
Name: ACT_DWELL_TIME_VISIT_MIN, Length: 453410, dtype: int64
ECR-2043      288
ECR-331472    288
ECR-353768    288
ECR-962662    288
ECR-412429    288
             ... 
ECR-587646     72
ECR-870068     72
ECR-527218     72
ECR-416019     72
ECR-919626     72
Name: ECR_ID, Length: 21126, dtype: int64
0.00     1173945
0.02        5298
0.03        2799
0.05        2733
0.01        1479
          ...   
4.24           3
2.57           3
21.11          3
81.25          3
8.21           3
Name: LOY_DWELL_TIME_USER_MIN, Length: 444976, dtype: int64
0.00    973446
1.00    312990
2.00   

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
2,ECR_ID,"[[nan, ECR-10122750, ECR-10233150, ECR-1039975...",21127,object,0.00
0,ACT_CLICK_DEPTH,"[[5.2995508141, 5.2091959899, 5.2162749669, 5....",248267,float64,0.00
1,ACT_DWELL_TIME_VISIT_MIN,"[[2.868268450933, 3.020499749131, 2.9650547119...",453410,float64,0.00
3,LOY_DWELL_TIME_USER_MIN,"[[3.150083932854, 3.3881751088459997, 3.315838...",444976,float64,0.00
4,LOY_RETURN_RATE,"[[1.0982528263, 1.1217266645000001, 1.11830608...",148857,float64,0.00
5,POP_ACTIVE_USERS,"[[29190, 31390, 28384, 32226, 31703, 39458, 38...",12798,int64,0.00
6,POP_PAGE_VIEWS,"[[169893, 183421, 165575, 190321, 189726, 2398...",47332,int64,0.00
7,POP_TIME_SPENT_HRS,"[[1532.5158333333331, 1772.580277777778, 1568....",252881,float64,0.00
8,POP_VISITS,"[[32058, 35211, 31742, 36237, 35546, 44198, 42...",21479,int64,0.00
9,PROD_NAME,"[[SCIVAL, SCIENCEDIRECT, REAXYS, SCOPUS]]",4,object,0.00


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

-------------------- Dropping Columns with missing data --------------------
Following 0 columns have missing data over the threshold and will be removed
Index([], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
2,ECR_ID,"[[nan, ECR-10122750, ECR-10233150, ECR-1039975...",21127,object,0.00
0,ACT_CLICK_DEPTH,"[[5.2995508141, 5.2091959899, 5.2162749669, 5....",248267,float64,0.00
1,ACT_DWELL_TIME_VISIT_MIN,"[[2.868268450933, 3.020499749131, 2.9650547119...",453410,float64,0.00
3,LOY_DWELL_TIME_USER_MIN,"[[3.150083932854, 3.3881751088459997, 3.315838...",444976,float64,0.00
4,LOY_RETURN_RATE,"[[1.0982528263, 1.1217266645000001, 1.11830608...",148857,float64,0.00
5,POP_ACTIVE_USERS,"[[29190, 31390, 28384, 32226, 31703, 39458, 38...",12798,int64,0.00
6,POP_PAGE_VIEWS,"[[169893, 183421, 165575, 190321, 189726, 2398...",47332,int64,0.00
7,POP_TIME_SPENT_HRS,"[[1532.5158333333331, 1772.580277777778, 1568....",252881,float64,0.00
8,POP_VISITS,"[[32058, 35211, 31742, 36237, 35546, 44198, 42...",21479,int64,0.00
9,PROD_NAME,"[[SCIVAL, SCIENCEDIRECT, REAXYS, SCOPUS]]",4,object,0.00


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

No data needs imputing in this case.

In [8]:
#Filling all missing values with 0
# data = data.fillna(0)

#Filling missing values with the most frequest values


# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = [])

Dropping outliers for ACT_CLICK_DEPTH upper limit = 11.814 and lower limit = 0.0
Dropping outliers for ACT_DWELL_TIME_VISIT_MIN upper limit = 12.436011080332 and lower limit = 0.35
Dropping outliers for LOY_DWELL_TIME_USER_MIN upper limit = 33.202918340027 and lower limit = 1.733021806853
Dropping outliers for LOY_RETURN_RATE upper limit = 3.6666666667000003 and lower limit = 1.1111111111
Dropping outliers for POP_ACTIVE_USERS upper limit = 2879.0 and lower limit = 5.0
Dropping outliers for POP_PAGE_VIEWS upper limit = 14662.0 and lower limit = 54.0
Dropping outliers for POP_TIME_SPENT_HRS upper limit = 208.31833333333302 and lower limit = 1.5475
Dropping outliers for POP_VISITS upper limit = 1442.0 and lower limit = 27.0


# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
10,REPORT_AGG,[[MONTH]],1,object,0.00
9,PROD_NAME,"[[SCIVAL, SCIENCEDIRECT, REAXYS, SCOPUS]]",4,object,0.00
11,REPORT_DT,"[[2017-12-01, 2018-01-01, 2018-02-01, 2018-03-...",24,object,0.00


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
10,REPORT_AGG,[[MONTH]],1,object,0.00
9,PROD_NAME,"[[SCIVAL, SCIENCEDIRECT, REAXYS, SCOPUS]]",4,object,0.00
11,REPORT_DT,"[[2017-12-01, 2018-01-01, 2018-02-01, 2018-03-...",24,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 8 features - **********************
 ACT_CLICK_DEPTH standardized to ACT_CLICK_DEPTH
 ACT_DWELL_TIME_VISIT_MIN standardized to ACT_DWELL_TIME_VISIT_MIN
 LOY_DWELL_TIME_USER_MIN standardized to LOY_DWELL_TIME_USER_MIN
 LOY_RETURN_RATE standardized to LOY_RETURN_RATE
 POP_ACTIVE_USERS standardized to POP_ACTIVE_USERS
 POP_PAGE_VIEWS standardized to POP_PAGE_VIEWS
 POP_TIME_SPENT_HRS standardized to POP_TIME_SPENT_HRS
 POP_VISITS standardized to POP_VISITS


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
# print(columns_to_bin)

In [18]:
# # Specify Columns to encode
# columns_to_exclude = ['REPORT_DT']
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [19]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [20]:
# df.shape

In [21]:
df.head()

,ACT_CLICK_DEPTH,ACT_DWELL_TIME_VISIT_MIN,ECR_ID,LOY_DWELL_TIME_USER_MIN,LOY_RETURN_RATE,POP_ACTIVE_USERS,POP_PAGE_VIEWS,POP_TIME_SPENT_HRS,POP_VISITS,PROD_NAME,REPORT_AGG,REPORT_DT
96,1.91,2.25,ECR-104069,0.78,-0.76,-0.83,-0.62,-0.65,-0.87,SCIVAL,MONTH,2017-12-01
97,1.02,0.34,ECR-104069,-0.47,-1.13,-0.83,-0.68,-0.74,-0.88,SCIVAL,MONTH,2018-01-01
98,0.77,1.80,ECR-104069,0.24,-1.11,-0.80,-0.67,-0.65,-0.86,SCIVAL,MONTH,2018-02-01
99,1.63,2.19,ECR-104069,1.63,0.16,-0.80,-0.52,-0.52,-0.79,SCIVAL,MONTH,2018-03-01
100,2.11,2.37,ECR-104069,1.74,0.14,-0.77,-0.44,-0.47,-0.77,SCIVAL,MONTH,2018-04-01


## Transform Date columns

Recency and Frequency Variables and length / duration.

We have Expected Close Date, but we don't have a Risk Opened Date so we might not be able to derive meaningful variables such as 'Time Risk is Open'.


In [22]:
# convert date columns to date data type
#N/A

In [23]:
date_columns = ['REPORT_DT']

In [24]:
# calculate new variable expected_subscription_length
#N/A

In [25]:
 df.head()

,ACT_CLICK_DEPTH,ACT_DWELL_TIME_VISIT_MIN,ECR_ID,LOY_DWELL_TIME_USER_MIN,LOY_RETURN_RATE,POP_ACTIVE_USERS,POP_PAGE_VIEWS,POP_TIME_SPENT_HRS,POP_VISITS,PROD_NAME,REPORT_AGG,REPORT_DT
96,1.91,2.25,ECR-104069,0.78,-0.76,-0.83,-0.62,-0.65,-0.87,SCIVAL,MONTH,2017-12-01
97,1.02,0.34,ECR-104069,-0.47,-1.13,-0.83,-0.68,-0.74,-0.88,SCIVAL,MONTH,2018-01-01
98,0.77,1.80,ECR-104069,0.24,-1.11,-0.80,-0.67,-0.65,-0.86,SCIVAL,MONTH,2018-02-01
99,1.63,2.19,ECR-104069,1.63,0.16,-0.80,-0.52,-0.52,-0.79,SCIVAL,MONTH,2018-03-01
100,2.11,2.37,ECR-104069,1.74,0.14,-0.77,-0.44,-0.47,-0.77,SCIVAL,MONTH,2018-04-01


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = []

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,ACT_CLICK_DEPTH,ACT_DWELL_TIME_VISIT_MIN,ECR_ID,LOY_DWELL_TIME_USER_MIN,LOY_RETURN_RATE,POP_ACTIVE_USERS,POP_PAGE_VIEWS,POP_TIME_SPENT_HRS,POP_VISITS,PROD_NAME,REPORT_AGG,REPORT_DT
96,1.91,2.25,ECR-104069,0.78,-0.76,-0.83,-0.62,-0.65,-0.87,SCIVAL,MONTH,2017-12-01
97,1.02,0.34,ECR-104069,-0.47,-1.13,-0.83,-0.68,-0.74,-0.88,SCIVAL,MONTH,2018-01-01
98,0.77,1.80,ECR-104069,0.24,-1.11,-0.80,-0.67,-0.65,-0.86,SCIVAL,MONTH,2018-02-01
99,1.63,2.19,ECR-104069,1.63,0.16,-0.80,-0.52,-0.52,-0.79,SCIVAL,MONTH,2018-03-01
100,2.11,2.37,ECR-104069,1.74,0.14,-0.77,-0.44,-0.47,-0.77,SCIVAL,MONTH,2018-04-01


Save clean pickle files

In [29]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['usage_file']+"_for_graphs.pickle")